<a href="https://colab.research.google.com/github/bhumilshah26/precision_agriculture_ml/blob/main/SmartCrop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/SmartCropMergedDataset.csv")

In [ ]:
df.head()

In [ ]:
print(df.shape)
print(df.columns)
print(df.head())


In [ ]:

# Check data types & missing values
print(df.info())
print(df.isnull().sum())

In [ ]:
# Drop rows with missing essential values
df = df.dropna(subset=['state_name', 'district_name', 'crop_year', 'season', 'crop', 'area_', 'production_'])

# Remove zero or negative area or production
df = df[(df['area_'] > 0) & (df['production_'] > 0)]

# Create a yield column (tons per hectare, assuming same units)
df['yield'] = df['production_'] / df['area_']

# Standardize text
for col in ['state_name', 'district_name', 'season', 'crop']:
    df[col] = df[col].str.strip().str.title()

print(df.head())


In [ ]:
print(df.isnull().sum())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Top 10 crops by total area
top_crops = df.groupby('crop')['area_'].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(10,5))
sns.barplot(x=top_crops.values, y=top_crops.index, palette="viridis")
plt.title("Top 10 Crops by Total Area")
plt.xlabel("Total Area")
plt.ylabel("Crop")
plt.show()

# Top 10 crops by production
top_prod = df.groupby('crop')['production_'].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(10,5))
sns.barplot(x=top_prod.values, y=top_prod.index, palette="mako")
plt.title("Top 10 Crops by Total Production")
plt.xlabel("Total Production")
plt.ylabel("Crop")
plt.show()


In [ ]:
# Year-wise total area & production
yearly = df.groupby('crop_year')[['area_', 'production_']].sum().reset_index()

plt.figure(figsize=(10,5))
sns.lineplot(data=yearly, x='crop_year', y='area_', label='Area', marker='o')
sns.lineplot(data=yearly, x='crop_year', y='production_', label='Production', marker='s')
plt.title("Yearly Trends of Total Area and Production")
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(data=df, x='season', order=df['season'].value_counts().index, palette='cubehelix')
plt.title("Seasonal Distribution of Records")
plt.xlabel("Season")
plt.ylabel("Count")
plt.show()

# Total production by season
season_prod = df.groupby('season')['production_'].sum().sort_values(ascending=False)
plt.figure(figsize=(6,4))
sns.barplot(x=season_prod.index, y=season_prod.values, palette="coolwarm")
plt.title("Total Production by Season")
plt.show()


In [ ]:
# Top 10 states by total production
state_prod = df.groupby('state_name')['production_'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(10,5))
sns.barplot(x=state_prod.values, y=state_prod.index, palette="crest")
plt.title("Top 10 States by Total Production")
plt.xlabel("Total Production")
plt.ylabel("State")
plt.show()


In [ ]:
# Average yield per crop
avg_yield = df.groupby('crop')['yield'].mean().sort_values(ascending=False).head(15)

plt.figure(figsize=(10,5))
sns.barplot(x=avg_yield.values, y=avg_yield.index, palette="flare")
plt.title("Top 15 Crops by Average Yield")
plt.xlabel("Average Yield (Production/Area)")
plt.ylabel("Crop")
plt.show()


In [ ]:
corr = df[['area_', 'production_', 'yield']].corr()
sns.heatmap(corr, annot=True, cmap='Blues')
plt.title("Correlation Heatmap")
plt.show()


In [ ]:
# Scatterplot to see if some yields are abnormally high
plt.figure(figsize=(8,5))
sns.scatterplot(data=df, x='area_', y='yield', alpha=0.5)
plt.xscale('log')
plt.yscale('log')
plt.title("Yield vs Area (log scale)")
plt.show()


In [ ]:
pip install earthengine-api geemap geopandas geopy

In [ ]:
import ee
ee.Authenticate()  # follow the link and approve access

# Replace 'your-project-id' below with your actual GEE project
ee.Initialize(project='smartcrop-476713')


In [ ]:
import ee
ee.Initialize(project='smartcrop-476713')

# Test: print first Sentinel-2 image date globally
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
first = s2.first()
print("Sentinel-2 bands:", first.bandNames().getInfo())
print("First image date:", first.date().format().getInfo())


In [ ]:
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
print("CHIRPS sample:", chirps.first().bandNames().getInfo())

smap = ee.ImageCollection("NASA_USDA/HSL/SMAP10KM_soil_moisture")
print("SMAP sample:", smap.first().bandNames().getInfo())


In [ ]:
# PIPELINE: Place -> lat/lon -> fetch GEE timeseries -> indicators -> assessment
# Run this in the same env where ee.Initialize(project='smartcrop-476713') already worked.

import ee
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
import seaborn as sns

ee.Initialize(project='smartcrop-476713')  # ensure authenticated

# -------------------------
# 1) Geocode place name
# -------------------------
def geocode_place(place_name, user_agent='myapp'):
    geolocator = Nominatim(user_agent=user_agent, timeout=10)
    loc = geolocator.geocode(place_name, addressdetails=True)
    if loc is None:
        raise ValueError(f"Place not found: {place_name}")
    return {'lat': loc.latitude, 'lon': loc.longitude, 'raw': loc.raw}

# -------------------------
# 2) Build buffer polygon (in meters)
# -------------------------
def buffer_point_geometry(lat, lon, radius_m=5000):
    pt = ee.Geometry.Point(lon, lat)
    poly = pt.buffer(radius_m)  # buffer in meters, Earth Engine uses meters
    return poly

# -------------------------
# 3) Fetch time-series from GEE for date range
#    Returns pandas DataFrame with daily (or aggregated) features
# -------------------------
def fetch_timeseries_gee(lat, lon, start_date, end_date, buffer_m=5000, daily=False):
    geom = buffer_point_geometry(lat, lon, buffer_m)
    start = ee.Date(start_date)
    end   = ee.Date(end_date)

    # Sentinel-2 NDVI (cloud-masked median per 5-day window to be robust)
    def s2_mask_clouds(img):
        # Keep it simple: use QA60 band if present; fallback no mask if not
        qa = img.select('QA60')
        mask = qa.lt(1)  # zero means no cloud bit flagged
        return img.updateMask(mask)

    s2col = (ee.ImageCollection("COPERNICUS/S2_SR")
            .filterDate(start, end)
            .filterBounds(geom)
            .map(lambda i: i.clip(geom))
            .map(lambda i: i.normalizedDifference(['B8','B4']).rename('NDVI'))
           )

    # We'll aggregate NDVI into weekly medians to avoid cloud noise
    def aggregate_weekly(col, start, end):
        # build list of weekly intervals
        start_dt = datetime.strptime(start, "%Y-%m-%d")
        end_dt   = datetime.strptime(end, "%Y-%m-%d")
        weeks = []
        d = start_dt
        while d < end_dt:
            wstart = ee.Date(d.strftime("%Y-%m-%d"))
            wend_dt = d + timedelta(days=7)
            wend = ee.Date(min(wend_dt.strftime("%Y-%m-%d"), end))
            weeks.append((wstart, wend))
            d = wend_dt
        imgs = []
        for (ws, we) in weeks:
            sub = col.filterDate(ws, we)
            # median NDVI for the window
            med = sub.median().set('system:time_start', ws.millis())
            imgs.append(med)
        return ee.ImageCollection(imgs)

    s2_weekly = aggregate_weekly(s2col, start_date, end_date)

    # MODIS LST (Terra MOD11A1 daily LST at 1km)
    modis = (ee.ImageCollection("MODIS/061/MOD11A1")
             .filterDate(start, end)
             .filterBounds(geom)
             .map(lambda i: i.select('LST_Day_1km').multiply(0.02).rename('LST'))  # scale 0.02 -> K, convert to C later
            )

    # CHIRPS precipitation daily (mm)
    chirps = (ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
              .filterDate(start, end)
              .filterBounds(geom)
              .select('precipitation')
             )

    # SMAP soil moisture product (NASA_USDA/HSL/SMAP10KM_soil_moisture) uses 'ssm'
    smap = (ee.ImageCollection("NASA_USDA/HSL/SMAP10KM_soil_moisture")
            .filterDate(start, end)
            .filterBounds(geom)
            .select('ssm')
           )

    # ERA5 daily 2m temperature and surface solar radiation downwards (ssrd)
    era5 = (ee.ImageCollection("ECMWF/ERA5/DAILY")
           .filterDate(start, end)
           .filterBounds(geom)
           )
    # era5 bands include 'mean_2m_air_temperature' (Kelvin), 'surface_solar_radiation_downwards' (J m-2)
    # if not present, use ERA5-Land collection name variations as available

    # helper to reduce collection into a pandas series keyed by date (system:time_start)
    def collection_to_timeseries(col, band_name, scale):
        def image_to_dict(img):
            date = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd')
            stat = img.reduceRegion(ee.Reducer.mean(), geom, scale=scale, bestEffort=True).get(band_name)
            return ee.Feature(None, {'date': date, band_name: stat})
        feats = col.map(image_to_dict).filter(ee.Filter.notNull([band_name]))
        # export as list
        features_list = feats.getInfo()['features']
        rows = []
        for f in features_list:
            d = f['properties']['date']
            v = f['properties'][band_name]
            rows.append((d, v))
        df = pd.DataFrame(rows, columns=['date', band_name])
        df['date'] = pd.to_datetime(df['date'])
        df[band_name] = pd.to_numeric(df[band_name], errors='coerce')
        df = df.sort_values('date').reset_index(drop=True)
        return df

    # Convert weekly NDVI collection to timeseries
    ndvi_df = collection_to_timeseries(s2_weekly, 'NDVI', scale=1000)  # scale 1km for speedy reduce
    lst_df  = collection_to_timeseries(modis, 'LST', scale=1000)
    precip_df = collection_to_timeseries(chirps, 'precipitation', scale=5566)
    smap_df   = collection_to_timeseries(smap, 'ssm', scale=10000)
    # ERA5: try to fetch mean_2m_air_temperature and surface_solar_radiation_downwards
    try:
        t2m_df = collection_to_timeseries(era5.select('mean_2m_air_temperature'), 'mean_2m_air_temperature', scale=10000)
        # Kelvin -> Celsius
        t2m_df['mean_2m_air_temperature'] = t2m_df['mean_2m_air_temperature'] - 273.15
    except Exception as e:
        print("ERA5 2m temp failed:", e)
        t2m_df = pd.DataFrame(columns=['date','mean_2m_air_temperature'])

    try:
        ssrd_df = collection_to_timeseries(era5.select('surface_solar_radiation_downwards'), 'surface_solar_radiation_downwards', scale=10000)
        # convert J/m2 per day to kWh/m2/day: 1 J = 2.77778e-7 kWh
        ssrd_df['ssrd_kwh_m2'] = ssrd_df['surface_solar_radiation_downwards'] * 2.77778e-7
    except Exception as e:
        print("ERA5 ssrd failed:", e)
        ssrd_df = pd.DataFrame(columns=['date','surface_solar_radiation_downwards','ssrd_kwh_m2'])

    # Merge timeseries on date using outer join (then forward/backfill small gaps)
    df = ndvi_df.merge(lst_df, on='date', how='outer') \
                .merge(precip_df, on='date', how='outer') \
                .merge(smap_df, on='date', how='outer') \
                .merge(t2m_df, on='date', how='outer') \
                .merge(ssrd_df[['date','ssrd_kwh_m2']], on='date', how='outer')

    # fill small gaps (interpolate)
    df = df.sort_values('date').reset_index(drop=True)
    df.interpolate(method='time', inplace=True, limit=3)
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)

    # convert LST (MODIS) from scaled Kelvin to Celsius if present
    if 'LST' in df.columns:
        # MODIS LST_Day_1km scaled by 0.02 -> Kelvin
        df['LST_C'] = df['LST'] * 0.02 - 273.15

    # ensure numeric types
    for c in df.columns:
        if c != 'date':
            df[c] = pd.to_numeric(df[c], errors='coerce')

    # optionally return daily vs weekly (we used weekly NDVI; you can resample)
    return df

# -------------------------
# 4) Indicators & heuristic assessment
# -------------------------
def compute_indicators(df):
    # assume df has 'date' and at least ndvi, ssm, precipitation, mean_2m_air_temperature, LST_C, ssrd_kwh_m2
    out = {}
    # rolling seasonal stats (choose 30-day window)
    df = df.set_index('date').sort_index()

    window = 30
    # median/mean levels
    out['ndvi_mean'] = df['NDVI'].mean() if 'NDVI' in df else np.nan
    out['ndvi_trend_slope'] = np.nan
    try:
        # trend slope (linear fit of NDVI over time)
        x = np.arange(len(df))
        y = df['NDVI'].values
        mask = ~np.isnan(y)
        if mask.sum() > 2:
            slope = np.polyfit(x[mask], y[mask], 1)[0]
            out['ndvi_trend_slope'] = slope
    except Exception:
        pass

    out['smap_mean'] = df['ssm'].mean() if 'ssm' in df else np.nan
    out['smap_trend'] = np.nan
    try:
        y = df['ssm'].values
        mask = ~np.isnan(y)
        if mask.sum() > 2:
            out['smap_trend'] = np.polyfit(x[mask], y[mask], 1)[0]
    except Exception:
        pass

    out['precip_total'] = df['precipitation'].sum() if 'precipitation' in df else np.nan
    out['precip_mean'] = df['precipitation'].mean() if 'precipitation' in df else np.nan
    out['t2m_mean'] = df['mean_2m_air_temperature'].mean() if 'mean_2m_air_temperature' in df else np.nan
    out['lst_mean'] = df['LST_C'].mean() if 'LST_C' in df else np.nan
    out['ssrd_mean_kwh_m2'] = df['ssrd_kwh_m2'].mean() if 'ssrd_kwh_m2' in df else np.nan

    # simple anomalies relative to rolling historical mean (if df contains multi-year, you can compute long-term)
    out['ndvi_anom'] = df['NDVI'].iloc[-1] - df['NDVI'].mean() if 'NDVI' in df else np.nan

    return out

def assess_crop_health(indicators):
    """
    Heuristic scoring. Positive score => improving, negative => depreciating.
    Weighted contributions:
      - NDVI trend (most important)
      - Soil moisture trend
      - Precipitation anomaly
      - Temperature stress (very high LST or T2M reduces score)
      - Solar radiation (gives energy)
    """
    score = 0.0
    # weights (hand-tuned — you can adjust)
    w_ndvi = 5.0
    w_smap = 3.0
    w_precip = 1.0
    w_temp = -0.8
    w_ssrd = 0.5

    ndvi_s = indicators.get('ndvi_trend_slope', 0) or 0
    smap_s = indicators.get('smap_trend', 0) or 0
    precip_total = indicators.get('precip_total', 0) or 0
    t2m = indicators.get('t2m_mean', np.nan)
    ssrd = indicators.get('ssrd_mean_kwh_m2', 0) or 0

    # Normalize signals roughly (ad-hoc):
    score += w_ndvi * (ndvi_s * 100)   # NDVI slope is small, scale it
    score += w_smap * (smap_s * 100)   # smap slope is small
    # precipitation: relative to 30-day mean, but we only have current; use total
    score += w_precip * math.log1p(precip_total)
    if not math.isnan(t2m):
        # penalize if mean temp > 35C OR <5C (crop dependent; this is generic)
        if t2m > 35:
            score += w_temp * (t2m - 35)
        elif t2m < 5:
            score += w_temp * (5 - t2m)
    score += w_ssrd * ssrd

    # map to label
    if score > 3:
        label = 'Improving'
    elif score < -3:
        label = 'Depreciating'
    else:
        label = 'Stable'

    return {'score': score, 'label': label}

# -------------------------
# 5) Full wrapper: place -> assessment
# -------------------------
def assess_place(place_name, start_date, end_date, buffer_m=5000):
    g = geocode_place(place_name)
    lat, lon = g['lat'], g['lon']
    print(f"Place: {place_name} -> lat {lat:.4f}, lon {lon:.4f}")
    ts = fetch_timeseries_gee(lat, lon, start_date, end_date, buffer_m=buffer_m)
    inds = compute_indicators(ts)
    result = assess_crop_health(inds)
    # attach some series for plotting
    result['timeseries'] = ts
    result['indicators'] = inds
    return result

# -------------------------
# Example usage
# -------------------------
if __name__ == "__main__":
    place = "Ahmedabad, India"
    # example: last 1 year
    end = datetime.utcnow().date()
    start = end - timedelta(days=365)
    res = assess_place(place, start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"), buffer_m=10000)
    print("Assessment label:", res['label'])
    print("Score:", res['score'])
    print("Key indicators:", res['indicators'])

    # Quick plot NDVI and soil moisture over time
    ts = res['timeseries']
    fig, ax = plt.subplots(2,1, figsize=(10,6), sharex=True)
    if 'NDVI' in ts:
        ax[0].plot(ts['date'], ts['NDVI'], marker='o', label='NDVI')
        ax[0].legend()
    if 'ssm' in ts:
        ax[1].plot(ts['date'], ts['ssm'], marker='o', label='Soil moisture (SMAP)')
        ax[1].legend()
    plt.tight_layout()
    plt.show()

# -------------------------
# End of pipeline
# -------------------------


In [ ]:
# --- Imports ---
import ee
import pandas as pd
from geopy.geocoders import Nominatim
from datetime import datetime
import numpy as np
import time

ee.Initialize(project='smartcrop-476713')

# --- Helper 1: Get lat/lon from district/state ---
geolocator = Nominatim(user_agent="smartcrop", timeout=10)

def get_latlon(state, district):
    query = f"{district}, {state}, India"
    try:
        loc = geolocator.geocode(query)
        if loc:
            return loc.latitude, loc.longitude
    except Exception:
        pass
    return np.nan, np.nan


# --- Helper 2: Fetch remote sensing seasonal features from GEE ---
def get_remote_features(lat, lon, crop_year, season):
    """Fetch seasonal averages from GEE for one point."""
    try:
        # convert to EE geometry
        geom = ee.Geometry.Point(lon, lat)

        # Define season windows (adjust if needed)
        if season.lower() == 'kharif':
            start = ee.Date.fromYMD(int(crop_year), 6, 1)
            end   = ee.Date.fromYMD(int(crop_year), 10, 31)
        elif season.lower() == 'rabi':
            start = ee.Date.fromYMD(int(crop_year), 10, 1)
            end   = ee.Date.fromYMD(int(crop_year)+1, 3, 31)
        else:  # whole year
            start = ee.Date.fromYMD(int(crop_year), 1, 1)
            end   = ee.Date.fromYMD(int(crop_year), 12, 31)

        # Sentinel-2 NDVI
        s2 = (ee.ImageCollection("COPERNICUS/S2_SR")
              .filterDate(start, end)
              .filterBounds(geom)
              .map(lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI')))
        ndvi_mean = s2.select('NDVI').mean().reduceRegion(
            reducer=ee.Reducer.mean(), geometry=geom, scale=1000, bestEffort=True).get('NDVI')

        # SMAP soil moisture
        smap = (ee.ImageCollection("NASA_USDA/HSL/SMAP10KM_soil_moisture")
                .filterDate(start, end)
                .filterBounds(geom)
                .select('ssm'))
        smap_mean = smap.mean().reduceRegion(
            ee.Reducer.mean(), geom, scale=10000, bestEffort=True).get('ssm')

        # CHIRPS precipitation
        chirps = (ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
                  .filterDate(start, end)
                  .filterBounds(geom)
                  .select('precipitation'))
        precip_sum = chirps.sum().reduceRegion(
            ee.Reducer.mean(), geom, scale=5566, bestEffort=True).get('precipitation')

        # ERA5 temperature and solar radiation
        era5 = (ee.ImageCollection("ECMWF/ERA5/DAILY")
                .filterDate(start, end)
                .filterBounds(geom))
        temp_mean = era5.select('mean_2m_air_temperature').mean().reduceRegion(
            ee.Reducer.mean(), geom, scale=10000, bestEffort=True).get('mean_2m_air_temperature')
        solar_mean = era5.select('surface_solar_radiation_downwards').mean().reduceRegion(
            ee.Reducer.mean(), geom, scale=10000, bestEffort=True).get('surface_solar_radiation_downwards')

        # Convert results to Python types
        result = {
            'ndvi_mean': ee.Number(ndvi_mean).getInfo() if ndvi_mean else np.nan,
            'soil_moisture': ee.Number(smap_mean).getInfo() if smap_mean else np.nan,
            'precip_total': ee.Number(precip_sum).getInfo() if precip_sum else np.nan,
            'temp_mean_c': (ee.Number(temp_mean).getInfo() - 273.15) if temp_mean else np.nan,
            'sunshine_kwh_m2': ee.Number(solar_mean).getInfo() * 2.77778e-7 if solar_mean else np.nan
        }
        return result

    except Exception as e:
        print(f"Error for ({lat},{lon},{crop_year},{season}):", e)
        return {'ndvi_mean': np.nan, 'soil_moisture': np.nan,
                'precip_total': np.nan, 'temp_mean_c': np.nan,
                'sunshine_kwh_m2': np.nan}


# --- Step 1: Prepare unique keys (district-season-year) ---
unique_keys = df[['state_name','district_name','crop_year','season']].drop_duplicates().reset_index(drop=True)

# Add lat/lon for each district (only once per state/district)
coords = []
for _, row in unique_keys.iterrows():
    lat, lon = get_latlon(row['state_name'], row['district_name'])
    coords.append((lat, lon))
    time.sleep(1)  # be nice to Nominatim
unique_keys['lat'], unique_keys['lon'] = zip(*coords)

# Drop those with missing coordinates
unique_keys = unique_keys.dropna(subset=['lat','lon']).reset_index(drop=True)

# --- Step 2: Fetch environmental features for each unique row ---
env_data = []
for i, row in unique_keys.head(5).iterrows():   # or .sample(5)
    print(f"[{i+1}/{len(unique_keys)}] {row['district_name']}, {row['crop_year']} {row['season']}")
    feats = get_remote_features(row['lat'], row['lon'], row['crop_year'], row['season'])
    env_data.append(feats)

env_df = pd.DataFrame(env_data)
merged_env = pd.concat([unique_keys, env_df], axis=1)

# --- Step 3: Merge back with main df ---
df_merged = df.merge(merged_env, on=['state_name','district_name','crop_year','season'], how='left')

# --- Step 4: Save for later ML analysis ---
df_merged.to_csv("cropdata_with_env_features.csv", index=False)
print("✅ Final dataset saved as cropdata_with_env_features.csv")

print(df_merged.head())


In [ ]:
# --- Imports ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import shap
import matplotlib.pyplot as plt
import seaborn as sns

# --- Load merged dataset ---
df = pd.read_csv("cropdata_with_env_features.csv")

# --- Basic cleaning ---
# Drop missing or invalid values
df = df.dropna(subset=['yield','ndvi_mean','soil_moisture','precip_total','temp_mean_c'])
df = df.replace([np.inf,-np.inf], np.nan).dropna()

# Quick look
print("Rows:", len(df))
print(df.head())


In [ ]:
# --- Encode categorical columns ---
df['season'] = df['season'].astype('category').cat.codes
df['crop'] = df['crop'].astype('category').cat.codes
df['state_name'] = df['state_name'].astype('category').cat.codes
df['district_name'] = df['district_name'].astype('category').cat.codes

# --- Feature selection ---
features = [
    'state_name', 'district_name', 'crop_year', 'season', 'crop',
    'area_', 'ndvi_mean', 'soil_moisture', 'precip_total',
    'temp_mean_c', 'sunshine_kwh_m2'
]
target = 'yield'

X = df[features]
y = df[target]


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=15,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)


In [ ]:
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R²: {r2:.3f}")
print(f"MAE: {mae:.3f}")
print(f"RMSE: {rmse:.3f}")

# Plot predicted vs actual yield
plt.figure(figsize=(6,6))
sns.scatterplot(x=y_test, y=y_pred)
plt.xlabel("Actual Yield")
plt.ylabel("Predicted Yield")
plt.title("Predicted vs Actual Yield")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.show()


In [ ]:
# --- Simple importance ---
importances = pd.Series(rf.feature_importances_, index=features).sort_values(ascending=False)
plt.figure(figsize=(8,5))
sns.barplot(x=importances.values, y=importances.index, palette="viridis")
plt.title("Feature Importance (Random Forest)")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()


In [ ]:
# Initialize SHAP explainer
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_test)

# --- Summary plot ---
shap.summary_plot(shap_values, X_test, feature_names=features)

# --- Dependence plot (example for NDVI) ---
shap.dependence_plot("ndvi_mean", shap_values, X_test, feature_names=features)


In [ ]:
# Suppose you have current-season environmental features for a district
new_sample = {
    'state_name': 10,  # encoded value or same as training mapping
    'district_name': 55,
    'crop_year': 2025,
    'season': 1,       # e.g. 0 = Kharif, 1 = Rabi
    'crop': 4,         # e.g. Rice
    'area_': 1500,
    'ndvi_mean': 0.52,
    'soil_moisture': 0.18,
    'precip_total': 740,
    'temp_mean_c': 30.5,
    'sunshine_kwh_m2': 5.1
}

sample_df = pd.DataFrame([new_sample])
pred_yield = rf.predict(sample_df)[0]
print(f"🌾 Predicted yield: {pred_yield:.2f} (units same as training data)")


rf = joblib.load("crop_yield_model.pkl")
